In [1]:
%pylab inline
import scipy.integrate as spi

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Datos Llantas tractivas 
rll=0.139 # Radio geométrico de las llantas en traseras (m)
r=0.98*rll # Radio efectivo llantas traseras (m)
rin=0.0631 #radio interno del neumático (m)

dll=0.975 # Distancia entre ejes de llantas delanteras (m)
alj=0.88 # Altura con el piloto sentado en el kart (estatura 1.68 m) (m)
rho=1.1955 # densidad del aire (kg/m^3)
af=(dll*alj)/2 # Area Frontal Kart (m^2)
cd=0.89 # Coeficiente de Arrastre Kart
vw=0.0 # velocidad del viento (m/s)
theta=0.0 # Angulo carretera
g=9.81 # Gravedad (m/s)

# Datos Transmisión
dp=11.0 # Dientes del piñon
dpl=39.0 # Dientes del plato
Nc=dpl/dp # Relación de Transmisión
ef=0.95 # Eficiencia de transmisión

# Datos eje llantas
reje=0.02 # Radio eje de las llantas (m)
meje=3.832 # Masa del eje (kg)

# Datos Kart general
masa=152.0 # masa del Kart (kg)
g=9.81 # Gravedad (m/s)
w=masa*g # peso del Kart (N)
m = masa

# Datos Inercias
Ie=5.2E-3 # Inercia motor de combustión (kg m^2) 

rpi=0.01525#(m)
rpla=0.04096 #(m)
rhoac=7850.0 # Densidad del Acero (kg/m^3)
bpi=0.01 # Ancho piñon (m)
bpla=0.02 # Ancho plato (m)
vpi= bpi*np.pi*rpi**2 # Volumen del piñon (m^3)
vpla=bpla*np.pi*rpla**2 # Volumen del plato (m^3)
mpi=rhoac*vpi # Masa del piñon (kg)
mpla=rhoac*vpla # Masa del plato (kg)
mllt=1.43 # Masa llantas traseras (kg)
mlld=1.20 # Masa llantas delanteras (kg)
rlld=0.129 # Radio llantas delanteras (m)
rind=0.0631 # Radio rin llantas delanteras (m)
mbt=0.7*mllt # El porcentaje de la masa total que pesa la parte "banda" de la llanta trasera
mdt=0.3*mllt # El porcentaje de la masa total que pesa la parte "disco" de la llanta trasera
mbd=0.7*mlld # El porcentaje de la masa total que pesa la parte "banda" de la llanta delantera
mdd=0.3*mlld # El porcentaje de la masa total que pesa la parte "disco" de la llanta delantera
illd=(mbd*rlld)+(mdd*(rlld**2-rind**2))/2 # Inercia 1 llanta delantera (kg-m^2)
illt=(mbt*rll)+(mdt*(rll**2-rin**2))/2 # Inercia 1 llanta trasera (kg-m^2)
ig=((mpi*(rpi**2))/2.0)+((mpla*(rpla**2))/2.0) # Inercia de la transmisión (kg-m^2)
ieje=(meje*reje**2)/2.0 # Inercia del eje de las llantas (kg-m^2)


Iw = illd + illt
Id = ig + ieje

In [3]:
# Condiciones iniciales
t_0=0
t_fin=10
t=np.linspace(t_0,t_fin,1000)
v_0=0.1
c_ini = [t_0,v_0] 

## Definición de fuerzas

In [4]:
def frodadura(vx,w):
    fr=0.01*(1+((vx/0.277778/100.0))) # Factor de rodadura la velocidad del kart debe estar en (km/h)
    return w*fr # (N)
def farrastre(vx):
    return (1.0/2.0)*rho*af*cd*(vx+vw)**2 # (N)
def torqueMotor(vx,r_eje,Nc):
    omega = (vx/r_eje)*Nc
    if(omega>5000):
        omega=5000
    rpm=omega*30/(np.pi)
    P=3.037*((-0.00000004564538677*rpm**3)+(0.0001056995459*rpm**2)+(1.53209191*rpm)-1.895523696)
    T=P/(rpm*2*np.pi/60)
    if(T>5.42):
        T=5.42
    return T
def Mr(T):
    mr=((Iw+Id)+Ie*Nc**2)/r**2
    return mr
def ax (velv,t0):
    xx,vel = velv 
    frod = frodadura(vel,w)
    fdrag = farrastre(vel)
    T = torqueMotor(vel,r,Nc)
    mr = Mr(T)
    ax = ((T*Nc*ef/r)-frod-fdrag)/(mr+m)
    y= vel,ax
    return y
def abr (velv,t0):
    xx,vel = velv
    fbr = (-20.5*ef*Nc)/r
    frod = frodadura(vel,w)
    fdrag = farrastre(vel)
    T = torqueMotor(vel,r,Nc)
    mr = Mr(T)
    abr = (fbr-frod-fdrag)/(mr+m)
    y = vel,abr
    return y
def acel(ax):
    t_a0 = 0.1  
    t_a = np.linspace(t_0,t_fin,1000) 
    sol_ax = spi.odeint(ax, c_ini, t_a) 
    vel = sol_ax[:,1]
    pos = sol_ax[:,0] 
    diff = vel[-1]-vel[len(vel)-2] 
    t_ai = t_fin
    while diff > 0.004: 
        t_ai = t_fin + 1 
        t_a = np.linspace(t_a0,t_ai,1000) 
        sol_ax = spi.odeint(ax, c_ini, t_a) 
        vel = sol_ax[:,1] 
        pos = sol_ax[:,0] 
        diff = vel[-1]-vel[len(vel)-2] 
    acel = np.gradient(vel)*100
    return (t_a,pos,vel,acel,t_ai)
def brake(abr,t_a,vel,pos,acel,t_ai):
    v_max = vel[-1] 
    x_1 = pos[-1] 
    c_fr = [x_1,v_max]
    t_a1 = t_ai
    t_a2 = t_a1 + 0.5 
    t_br = np.linspace(t_a1,t_a2,100) 
    sol_br = spi.odeint(abr, c_fr, t_br) 
    v_br = sol_br[:,1]
    x_br = sol_br[:,0] 

    while v_br[-1] > 0.1: 
        t_a2 = t_a2 + 0.5 
        t_br = np.linspace(t_a1,t_a2,100) 
        sol_br = spi.odeint(abr, c_fr, t_br) 
        v_br = sol_br[:,1] 
        x_br = sol_br[:,0] 
    acelbr =np.gradient(v_br)*100
    t = np.concatenate((t_a,t_br))
    x = np.concatenate((pos,x_br))
    v = np.concatenate((vel,v_br))
    a = np.concatenate((acel,acelbr))
    return (t,x,v,a)

In [ ]:
soln1m1= spi.odeint(ax,c_ini,t)
an1m1= acel(ax)
brn1m1= brake(abr,an1m1[0],an1m1[1],an1m1[2],an1m1[3],an1m1[-1])

In [ ]:
soln1m2= spi.odeint(ax,c_ini,t)
an1m2= acel(ax)
brn1m2= brake(abr,an1m2[0],an1m2[1],an1m2[2],an1m2[3],an1m2[-1])

In [ ]:
soln1m3= spi.odeint(ax,c_ini,t)
an1m3= acel(ax)
brn1m3= brake(abr,an1m3[0],an1m3[1],an1m3[2],an1m3[3],an1m3[-1])

In [ ]:
soln2m1= spi.odeint(ax,c_ini,t)
an2m1= acel(ax)
brn2m1= brake(abr,an2m1[0],an2m1[1],an2m1[2],an2m1[3],an2m1[-1])

In [ ]:
soln2m2= spi.odeint(ax,c_ini,t)
an2m2= acel(ax)
brn2m2= brake(abr,an2m2[0],an2m2[1],an2m2[2],an2m2[3],an2m2[-1])

In [ ]:
soln2m3= spi.odeint(ax,c_ini,t)
an2m3= acel(ax)
brn2m3= brake(abr,an2m3[0],an2m3[1],an2m3[2],an2m3[3],an2m3[-1])

In [ ]:
soln3m1= spi.odeint(ax,c_ini,t)
an3m1= acel(ax)
brn3m1= brake(abr,an3m1[0],an3m1[1],an3m1[2],an3m1[3],an3m1[-1])

In [ ]:
soln3m2= spi.odeint(ax,c_ini,t)
an3m2= acel(ax)
brn3m2= brake(abr,an3m2[0],an3m2[1],an3m2[2],an3m2[3],an3m2[-1])

In [ ]:
soln3m3= spi.odeint(ax,c_ini,t)
an3m3= acel(ax)
brn3m3= brake(abr,an3m3[0],an3m3[1],an3m3[2],an3m3[3],an3m3[-1])

In [ ]:
plt.plot(brn1m1[0],brn1m1[2],label='100 kg')
plt.plot(brn1m2[0],brn1m2[2],label='152 kg')
plt.plot(brn1m3[0],brn1m3[2],label='200 kg')
plt.xlabel('Tiempo [s]')
plt.ylabel('Velocidad [m/s]')
plt.title('V vs T')
plt.grid()
plt.legend(loc='best')
plt.show()